# 1. Load Dependencies

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from sys import getsizeof
import ast
import os
import joblib

!pip install -q -U torch watermark
%reload_ext watermark

!pip install scikit-tensor-py3
from sktensor import dtensor, cp_als

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline

ModuleNotFoundError: No module named 'google'

In [15]:
%watermark -v -p pandas,numpy,sklearn,matplotlib,seaborn,sktensor

UsageError: Line magic function `%watermark` not found.


# 2. Helper Functions

In [21]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


def show_memory(unit='MB', threshold='MB'):
    '''check memory usage

    :param unit: memory unit, `B`,`KB`,`MB`,`GB`
    :param threshold: only show memory usage greater than the threshold
    '''

    scale = {'B': 1, 'KB': 1024, 'MB': 1048576, 'GB': 1073741824}
    for i in list(globals().keys()):
        memory = eval('getsizeof({})'.format(i))
        if memory >= scale[threshold]:
            print(i, str(memory//scale[unit]) + ' ' + unit)

# 3. Load Data

In [22]:
# Read data

file_path_train = '/content/drive/My Drive/Colab Notebooks/DATA5703/Data/Dataset/TrainData_1021.csv'
file_path_test = '/content/drive/My Drive/Colab Notebooks/DATA5703/Data/Dataset/TestData_1021.csv'

train_df = pd.read_csv(file_path_train)
test_df = pd.read_csv(file_path_test)

data_df = pd.concat([train_df, test_df], ignore_index=True)

data_df.head(1)

NameError: name 'pd' is not defined

In [ ]:
print(data_df.shape)
show_memory()

: 

In [ ]:
# sequence data was stored in string formatted list
# convert sequence data to array

def str_to_arr(str_ls):
    return np.fromstring(str_ls[1:-1], dtype=np.float, sep=',')


start = time.time()

data_df['heart_rate'] = data_df.apply(lambda x: str_to_arr(x['heart_rate']), axis=1)
data_df['altitude'] = data_df.apply(lambda x: str_to_arr(x['altitude']), axis=1)
data_df['derived_speed'] = data_df.apply(lambda x: str_to_arr(x['derived_speed']), axis=1)
data_df['derived_distance'] = data_df.apply(lambda x: str_to_arr(x['derived_distance']), axis=1)
data_df['timestamp'] = data_df.apply(lambda x: str_to_arr(x['timestamp']), axis=1)

elapsed = format_time(time.time() - start)
print(elapsed)

data_df.describe()

NameError: name 'time' is not defined

# 4. Min-Max scaling

In [ ]:
# Load Min-max scaler models

scaler_dic = dict()
features = ['calories', 'distance', 'duration', 'heart_rate',
            'time_elapsed', 'altitude', 'derived_distance', 'speed']

dir = '/content/drive/My Drive/Colab Notebooks/DATA5703/scaler_model/'

for feature in features:
    path = os.path.join(dir, 'scaler_'+feature+'_2.m')
    scaler_dic[feature] = joblib.load(path)

scaler_dic

NameError: name 'os' is not defined

In [ ]:
sequence_df = data_df[['altitude', 'heart_rate',
                       'derived_distance', 'derived_speed']].copy()
sequence_df.rename(columns={'derived_speed': 'speed'}, inplace=True)

context_df = data_df[['userId', 'gender', 'sport',
                      'duration', 'calories', 'distance', 'Route_id']].copy()

display(sequence_df.head(1))
display(context_df.head(1))

: 

In [ ]:
# Apply min max scaler for each feature

start = time.time()

# for features in sequence_df
for feature in sequence_df.columns:
    # Reshape data using array.reshape(-1, 1) if data has a single feature
    sequence_df[feature] = sequence_df.apply(lambda x: np.concatenate(scaler_dic[feature].transform(x[feature].reshape(-1, 1)), axis=0)[0:499], axis=1)

# for features in context_df
for feature in context_df[context_df.columns.difference(['userId', 'gender', 'sport', 'Route_id'])].columns:
    # Reshape data using array.reshape(-1, 1) if data has a single feature
    context_df[feature] = scaler_dic[feature].transform(context_df[feature].to_numpy().reshape(-1, 1))


elapsed = format_time(time.time() - start)
print(elapsed)

display(sequence_df.head(1))
display(context_df.head(1))

: 

In [ ]:
data_df = pd.concat([context_df, sequence_df, data_df[['timestamp']]], axis=1)

data_df.head(1)

: 

# 5. Process data to construct User-Workout Route-Context Tensor

In [24]:
# Construct user profile, which will be used in embedding visualisation

# Count workout numbers and workout frequency for each user

def get_freq_from_ts(ts_list):

    deltas = []

    ts_list.sort()
    for i, ts in enumerate(ts_list):
        if i == 0:
            pass
        else:
            ts_prev = ts_list[i-1]

            dt_prev = datetime.datetime.fromtimestamp(ts_prev).date()
            dt = datetime.datetime.fromtimestamp(ts).date()

            deltas.append(abs(dt-dt_prev).days)
    try:
        if (len(deltas) == 1) & (sum(deltas) == 0):
            return 0
        else:
            return 1./(sum(deltas)/len(deltas))
    except:
        print('deltas: ', deltas)
        raise

start = time.time()

userlist = data_df.userId.unique().tolist()
print('There are {} users.'.format(len(userlist)))

user_profile = []

for user in userlist:
    time_stamps_list = data_df[data_df.userId == user].timestamp.tolist()
    time_stamp_list = [time_stamps[0] for time_stamps in time_stamps_list]
    gender = data_df[data_df.userId == user].gender.iloc[0]
    records = len(time_stamp_list)
    if records > 1:
        user_profile.append([user,records, get_freq_from_ts(time_stamp_list)])
    else:
        user_profile.append([user, 1,0])

user_profile_df = pd.DataFrame(user_profile, columns = ['userId','workout_records', 'workout_freq'])

elapsed = format_time(time.time() - start)
print(elapsed)

user_profile_df.head(1)

NameError: name 'time' is not defined

In [ ]:
# Compute avg_calories, avg_speed, avg_distance for each user

user_profile_list = []

for user in userlist:

    avg_calories = data_df[data_df.userId == user].calories.mean()
    avg_distance = data_df[data_df.userId == user].distance.mean()
    avg_speed = data_df[data_df.userId == user].apply(lambda x: x.speed.mean(), axis=1).mean()

    user_arr = np.array([user, avg_calories,avg_speed,avg_distance])

    user_profile_list.append(user_arr)

user_df = pd.DataFrame(user_profile_list, columns=['userId', 'avg_calories','avg_speed','avg_distance'])
user_df.head(1)

: 

In [ ]:
# Add additional features to user_profile_df

user_profile_df = user_profile_df.join(user_df.set_index('userId'), how='left', on='userId')

user_profile_df.head(1)

: 

In [ ]:
# Add workout numbers and workout frequency to data_df
start = time.time()

data_df['workout_records'] = data_df.apply(lambda x: user_profile_df[user_profile_df.userId == x.userId].workout_records.values[0], axis=1)
data_df['workout_freq'] = data_df.apply(lambda x: user_profile_df[user_profile_df.userId == x.userId].workout_freq.values[0], axis=1)

elapsed = format_time(time.time() - start)
print(elapsed)

data_df.head(1)

: 

In [ ]:
# Scale workout_records and workout_freq

scaler_dic = {}

scaler_workout_records = MinMaxScaler()
scaler_workout_freq = MinMaxScaler()

scaler_workout_records.fit(data_df.workout_records.to_numpy().reshape(-1, 1))
scaler_workout_freq.fit(data_df.workout_freq.to_numpy().reshape(-1, 1))

data_df['workout_records'] = scaler_workout_records.transform(data_df.workout_records.to_numpy().reshape(-1, 1)).reshape(1, -1)[0]
data_df['workout_freq'] = scaler_workout_freq.transform(data_df.workout_freq.to_numpy().reshape(-1, 1)).reshape(1, -1)[0]

: 

In [ ]:
# Compute some parameters from sequence data

start = time.time()

data_df['min_altitude'] = data_df.apply(lambda x: np.amin(x.altitude), axis=1)
data_df['avg_altitude'] = data_df.apply(lambda x: x.altitude.mean(), axis=1)
data_df['max_altitude'] = data_df.apply(lambda x: np.amax(x.altitude), axis=1)

data_df['min_heart_rate'] = data_df.apply(lambda x: np.amin(x.heart_rate), axis=1)
data_df['avg_heart_rate'] = data_df.apply(lambda x: x.heart_rate.mean(), axis=1)
data_df['max_heart_rate'] = data_df.apply(lambda x: np.amax(x.heart_rate), axis=1)

data_df['min_distance'] = data_df.apply(lambda x: np.amin(x.derived_distance), axis=1)
data_df['avg_distance'] = data_df.apply(lambda x: x.derived_distance.mean(), axis=1)
data_df['max_distance'] = data_df.apply(lambda x: np.amax(x.derived_distance), axis=1)

data_df['min_speed'] = data_df.apply(lambda x: np.amin(x.speed), axis=1)
data_df['avg_speed'] = data_df.apply(lambda x: x.speed.mean(), axis=1)
data_df['max_speed'] = data_df.apply(lambda x: np.amax(x.speed), axis=1)

elapsed = format_time(time.time() - start)
print(elapsed)

: 

In [ ]:
# Construct route profile, which will be used in embedding visualisation

route_list = data_df.Route_id.unique()
route_profile_list = []

for route in route_list:
    tmp_df = data_df[data_df.Route_id == route].copy()
    avg_distance_route = tmp_df.avg_distance.mean()
    avg_speed_route = tmp_df.avg_speed.mean()
    avg_calories_route = tmp_df.calories.mean()
    sport = tmp_df.sport.iloc[0]

    route_arr = np.array([route,sport,avg_calories_route,avg_speed_route,avg_distance_route])

    route_profile_list.append(route_arr)

route_profile_df = pd.DataFrame(route_profile_list, columns=['Route_id','sport','avg_calories','avg_speed','avg_distance'])

route_profile_df['Route_id'] = route_profile_df['Route_id'].astype(int)
route_profile_df[['avg_calories', 'avg_speed', 'avg_distance']] = route_profile_df[['avg_calories', 'avg_speed', 'avg_distance']].apply(pd.to_numeric)
route_profile_df.head(1)

: 

In [ ]:
# Encode gender feature

conditions = [(data_df['gender'] == 'male'),(data_df['gender'] == 'female'),(data_df['gender'] == 'unknown')]

choices = [0, 1, 2]

data_df['genderId'] = np.select(conditions, choices, default=0)

: 

In [ ]:
# Encode sport feature

conditions = [(data_df['sport'] == 'run'),(data_df['sport'] == 'bike'),(data_df['sport'] == 'mountain bike')]

choices = [0, 1, 2]

data_df['sportId'] = np.select(conditions, choices, default=0)

: 

In [ ]:
data_df.head(1)

: 

In [ ]:
# Compute user - workout route - context tensor

user_list = data_df.userId.unique()


start = time.time()

users_routes_list = []
route_idx = {}

for route in route_list:
    route_idx[route] = data_df[data_df.Route_id == route].sport.tolist()[0]

# # mask array for tensor decomposition
#     # array of booleans with the same shape as tensor
#     # 0 where the values are missing and 1 everywhere else
# mask_arr = np.ones((len(user_list), len(route_list), 13))

for i, user in enumerate(user_list):
    user_records = data_df[data_df.userId == user].shape[0]
    user_routes_list = []

    for j, route in enumerate(route_list):

        user_route_df = data_df[(data_df.userId == user) & (
            data_df.Route_id == route)].copy()
        # if this user has more than one workout in this route
        if user_route_df.shape[0] > 0:

            user_gender = user_route_df.genderId.tolist()[0]
            sport = user_route_df.sportId.tolist()[0]
            workout_records = user_route_df.workout_records.tolist()[0]
            workout_freq = user_route_df.workout_freq.tolist()[0]

            cw_min_duration = user_route_df.duration.min()
            cw_avg_duration = user_route_df.duration.mean()
            cw_max_duration = user_route_df.duration.max()

            cw_min_calories = user_route_df.calories.min()
            cw_avg_calories = user_route_df.calories.mean()
            cw_max_calories = user_route_df.calories.max()

            cw_min_distance = user_route_df.distance.min()
            cw_avg_distance = user_route_df.distance.mean()
            cw_max_distance = user_route_df.distance.max()

            cw_min_min_altitude = user_route_df.min_altitude.min()
            cw_avg_min_altitude = user_route_df.min_altitude.mean()
            cw_max_min_altitude = user_route_df.min_altitude.max()

            cw_min_avg_altitude = user_route_df.avg_altitude.min()
            cw_avg_avg_altitude = user_route_df.avg_altitude.mean()
            cw_max_avg_altitude = user_route_df.avg_altitude.max()

            cw_min_max_altitude = user_route_df.max_altitude.min()
            cw_avg_max_altitude = user_route_df.max_altitude.mean()
            cw_max_max_altitude = user_route_df.max_altitude.max()

            cw_min_min_heart_rate = user_route_df.min_heart_rate.min()
            cw_avg_min_heart_rate = user_route_df.min_heart_rate.mean()
            cw_max_min_heart_rate = user_route_df.min_heart_rate.max()

            cw_min_avg_heart_rate = user_route_df.avg_heart_rate.min()
            cw_avg_avg_heart_rate = user_route_df.avg_heart_rate.mean()
            cw_max_avg_heart_rate = user_route_df.avg_heart_rate.max()

            cw_min_max_heart_rate = user_route_df.max_heart_rate.min()
            cw_avg_max_heart_rate = user_route_df.max_heart_rate.mean()
            cw_max_max_heart_rate = user_route_df.max_heart_rate.max()

            cw_min_min_distance = user_route_df.min_distance.min()
            cw_avg_min_distance = user_route_df.min_distance.mean()
            cw_max_min_distance = user_route_df.min_distance.max()

            cw_min_avg_distance = user_route_df.avg_distance.min()
            cw_avg_avg_distance = user_route_df.avg_distance.mean()
            cw_max_avg_distance = user_route_df.avg_distance.max()

            cw_min_max_distance = user_route_df.max_distance.min()
            cw_avg_max_distance = user_route_df.max_distance.mean()
            cw_max_max_distance = user_route_df.max_distance.max()

            cw_min_min_speed = user_route_df.min_speed.min()
            cw_avg_min_speed = user_route_df.min_speed.mean()
            cw_max_min_speed = user_route_df.min_speed.max()

            cw_min_avg_speed = user_route_df.avg_speed.min()
            cw_avg_avg_speed = user_route_df.avg_speed.mean()
            cw_max_avg_speed = user_route_df.avg_speed.max()

            cw_min_max_speed = user_route_df.max_speed.min()
            cw_avg_max_speed = user_route_df.max_speed.mean()
            cw_max_max_speed = user_route_df.max_speed.max()
            
            # proportion of this route in all workout route records of this user
            cw_route_weight = user_route_df.shape[0]/user_records

            user_route_arr = np.array([user_gender,
                                       sport,
                                       workout_records,
                                       workout_freq,
                                       cw_min_duration,
                                       cw_avg_duration,
                                       cw_max_duration,
                                       cw_min_calories,
                                       cw_avg_calories,
                                       cw_max_calories,
                                       cw_min_distance,
                                       cw_avg_distance,
                                       cw_max_distance,
                                       cw_min_min_altitude,
                                       cw_avg_min_altitude,
                                       cw_max_min_altitude,
                                       cw_min_avg_altitude,
                                       cw_avg_avg_altitude,
                                       cw_max_avg_altitude,
                                       cw_min_max_altitude,
                                       cw_avg_max_altitude,
                                       cw_max_max_altitude,
                                       cw_min_min_heart_rate,
                                       cw_avg_min_heart_rate,
                                       cw_max_min_heart_rate,
                                       cw_min_avg_heart_rate,
                                       cw_avg_avg_heart_rate,
                                       cw_max_avg_heart_rate,
                                       cw_min_max_heart_rate,
                                       cw_avg_max_heart_rate,
                                       cw_max_max_heart_rate,
                                       cw_min_min_distance,
                                       cw_avg_min_distance,
                                       cw_max_min_distance,
                                       cw_min_avg_distance,
                                       cw_avg_avg_distance,
                                       cw_max_avg_distance,
                                       cw_min_max_distance,
                                       cw_avg_max_distance,
                                       cw_max_max_distance,
                                       cw_min_min_speed,
                                       cw_avg_min_speed,
                                       cw_max_min_speed,
                                       cw_min_avg_speed,
                                       cw_avg_avg_speed,
                                       cw_max_avg_speed,
                                       cw_min_max_speed,
                                       cw_avg_max_speed,
                                       cw_max_max_speed, 
                                       cw_route_weight])
        else:
            user_route_arr = np.zeros([50])
            # mask_arr[i][j] = np.zeros([50])

        user_routes_list.append(user_route_arr)

    user_routes_arr = np.stack(user_routes_list, axis=0)
    users_routes_list.append(user_routes_arr)

users_routes_arr = np.stack(users_routes_list, axis=0)

elapsed = format_time(time.time() - start)
print(elapsed)

: 

# 6. CP Decomposition

## 6.1 Define function to perform CP Decomposition and evaluate number of CP ranks

The theories of CORCONDIA and Efficient CORCONDIA algorithms in this section are referenced from

1. Bro, R. and Kiers, H.A.L. (2003), A new efficient method for determining the number of components in PARAFAC models. J. Chemometrics, 17: 274-286. doi:10.1002/cem.801.

2. E. E. Papalexakis and C. Faloutsos, "Fast efficient and scalable Core Consistency Diagnostic for the parafac decomposition for big sparse tensors," 2015 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP), Brisbane, QLD, 2015, pp. 5441-5445, doi: 10.1109/ICASSP.2015.7179011.

3. Anna Sapienza (2017). Tensor decomposition techniques for analysing time-varying networks. Available at : http://porto.polito.it/2668112/. doi:10.6092/polito/porto/2668112.

**Core Consistency Diagnostic (CORCONDIA):**


CP Decomposition for a three-way tensor $\mathcal{X} \in \mathbb{R}^{I \times J \times K}$ can be represented as 

$$
\mathcal{X}=[[ \mathbf{A}, \mathbf{B}, \mathbf{C} ]]+\mathcal{E}
$$

that can be written in the matricised form

$$
\mathrm{X}=\mathrm{A}(\mathrm{C} \odot \mathrm{B})^{T}+\mathrm{E}
$$

where

* $X \in \mathbb{R}^{I \times JK}$: Original tensor to be decomposed in matrix form 
* $A \in \mathbb{R}^{I \times R}$: Can be seen as the user matrix after CP Decomposition
* $B \in \mathbb{R}^{J \times R}$: Can be seen as the route matrix after CP Decomposition
* $C \in \mathbb{R}^{K \times R}$: Can be seen as the contextual feature matrix after CP Decomposition
* $E \in \mathbb{R}^{I \times JK}$: Decomposition Error

To compute the core consistency value we first re-write the above equation as a Tucker3 model by adding a superdiagonal core tensor $\mathcal{L} \in \mathbb{R}^{R \times R \times R}$ with superdiagonal values of **ones** in its matricised form $\mathbf{L} \in \mathbb{R}^{R \times R R}$:

$$
\mathrm{X}=\mathrm{A} \mathbf{L} (\mathrm{C} \odot \mathrm{B})^{T}+\mathrm{E}
$$

Let's assume we have done the CP Decomposition with selected rank and learned matrices $\mathrm{A}$, $\mathrm{B}$ and $\mathrm{C}$. Then we fit the full Tucker3 model to the data by using the CP Decomposition matrices $\mathrm{A}$, $\mathrm{B}$ and $\mathrm{C}$, by minimising

$$
\sigma(\mathbf{G})=\left\|\mathbf{X}-\mathbf{A G}(\mathbf{C} \otimes \mathbf{B})^{\mathrm{T}}\right\|_{\mathrm{F}}^{2}
$$

over core matrix $\mathbf{G}$. This is essentially saying the residual decomposition error $\sigma(\mathbf{G})$ is minimised.

The optimal $\mathbf{G}$ can be computed by writing the equation in its vectorised form and applying a least squares algorithm

$$
\sigma(\mathbf{G})=\|\operatorname{vec} \mathbf{X}-(\mathbf{C} \otimes \mathbf{B} \otimes \mathbf{A}) \operatorname{vec} \mathbf{G}\|_{F}^{2}
$$

where $\otimes$ is **Kronecker product** (https://en.wikipedia.org/wiki/Kronecker_product).

The optimal $\mathbf{G}$ in the above equation can be determined as 

$$
\operatorname{vec} \mathbf{G}=(\mathbf{C} \otimes \mathbf{B} \otimes \mathbf{A})^{+} \operatorname{vec} \mathbf{X}
$$

when the residual decomposition error $\sigma(\mathbf{G})$ is minimised. Note that the superscript $+$ indicates a pseudoinverse.

The underlying idea is to find the similarity between $\mathcal{L}$ and $\mathcal{G}$ where $\mathcal{L}$ is a superdiagonal core tensor that all its superdiagonal values are 1 and all its off-superdiagonal values are 0. To compare the similarity, we can have a look at the distribution of the elements in the superdiagonal and off-superdiagonal of $\mathcal{G}$. If the superdiagonal elements of $\mathcal{G}$ are all close to the corresponding elements of $\mathcal{L}$, which is 1, and the off-superdiagonal elements of of $\mathcal{G}$ are all close to the corresponding elements of $\mathcal{L}$, which is 0, then we say the CP Decomposition result is appropriate.

Formally, the similarity between $\mathcal{L}$ and $\mathcal{G}$, or **`core consistency`** is given by

$$
c c=100\left(1-\frac{\sum{_{l=1}^{R}} \sum{_{m=1}^{R}} \sum{_{n=1}^{R}}\left(g_{l m n}-\lambda_{l m n}\right)^{2}}{R}\right)
$$

**Efficient Core Consistency (Efficient CORCONDIA):**

A faster way to compute the pseudo-inverse $(\mathbf{C} \otimes \mathbf{B} \otimes \mathbf{A})^{+}$ is 

$$
\left(\mathbf{V}_{a} \otimes \mathbf{V}_{b} \otimes \mathbf{V}_{c}\right)\left(\Sigma_{a}^{-1} \otimes \Sigma_{b}^{-1} \otimes \Sigma_{c}^{-1}\right)\left(\mathbf{U}_{a}^{T} \otimes \mathbf{U}_{b}^{T} \otimes \mathbf{U}_{c}^{T}\right)
$$

by performing $SVD$ where

* $\mathbf{A} = \mathbf{U}_{a} \Sigma_{a} \mathbf{V}_{a}^T$
* $\mathbf{B} = \mathbf{U}_{b} \Sigma_{b} \mathbf{V}_{b}^T$
* $\mathbf{C} = \mathbf{U}_{c} \Sigma_{c} \mathbf{V}_{c}^T$

Therefore, we have

$$
\operatorname{vec} \mathbf{G}=\left(\mathbf{V}_{a} \otimes \mathbf{V}_{b} \otimes \mathbf{V}_{c}\right)\left(\Sigma_{a}^{-1} \otimes \Sigma_{b}^{-1} \otimes \Sigma_{c}^{-1}\right)\left(\mathbf{U}_{a}^{T} \otimes \mathbf{U}_{b}^{T} \otimes \mathbf{U}_{c}^{T}\right) \operatorname{vec} \mathbf{X}
$$

This section of code was largely borrowed from https://github.com/alessandrobessi/corcondia

In [ ]:
def kronecker(matrices, tensor):
    K = len(matrices)
    x = tensor
    for k in range(K):
        M = matrices[k]
        # sktensor.core.ttm: Tensor times matrix product
        x = x.ttm(M, k)
    return x


def cp_with_eff_corcondia(tensor, num_comp=1):
    '''
    Parameters
    ----------
    num_comp: number of CP Decomposition components
    tensor: tensor to be decomposed
    '''

    # perfrom CP Decomposition
    kruskal_tensor, _, _ = cp_als(dtensor(tensor), num_comp, init='random')

    A = kruskal_tensor.U[0]
    B = kruskal_tensor.U[1]
    C = kruskal_tensor.U[2]

    # original tensor
    x = kruskal_tensor.totensor()

    # Compute Core array G with Efficient Core Consistency (efficient CORCONDIA)

    # 1. Singular Value Decomposition of the paramater matrices
    Ua, Sigma_a, Va = np.linalg.svd(A)
    Ub, Sigma_b, Vb = np.linalg.svd(B)
    Uc, Sigma_c, Vc = np.linalg.svd(C)

    # 2. Compute Pseudo-inverse of Sigma
    Sigma_a_Inverse = np.zeros((Ua.shape[0], Va.shape[0]), float)
    np.fill_diagonal(Sigma_a_Inverse, Sigma_a)

    Sigma_b_Inverse = np.zeros((Ub.shape[0], Vb.shape[0]), float)
    np.fill_diagonal(Sigma_b_Inverse, Sigma_b)

    Sigma_c_Inverse = np.zeros((Uc.shape[0], Vc.shape[0]), float)
    np.fill_diagonal(Sigma_c_Inverse, Sigma_c)

    Sigma_a_Inverse = np.linalg.pinv(Sigma_a_Inverse)
    Sigma_b_Inverse = np.linalg.pinv(Sigma_b_Inverse)
    Sigma_c_Inverse = np.linalg.pinv(Sigma_c_Inverse)

    # 3. Computer Core Matrix G
    y = kronecker([Ua.transpose(), Ub.transpose(), Uc.transpose()], x)
    z = kronecker([Sigma_a_Inverse, Sigma_b_Inverse, Sigma_c_Inverse], y)
    G = kronecker([Va.transpose(), Vb.transpose(), Vc.transpose()], z)

    # Compute Core Consistency

    # 1. Construct L (superdiagonal array of ones and off-superdiagonal of zeros)

    L = np.full((num_comp, num_comp, num_comp), 0)
    for i in range(num_comp):
        for j in range(num_comp):
            for l in range(num_comp):
                if i == j == l:
                    L[i][j][l] = 1

    # 2. Compute G subtract L

    c = 0
    for i in range(num_comp):
        for j in range(num_comp):
            for l in range(num_comp):
                c += float(G[i][j][l] - L[i][j][l]) ** 2.0

    # 3. Compute Core Consistency
    cc = 100 * (1 - (c / float(num_comp)))

    return A, B, C, round(cc)

: 

## 6.2 Evaluate number of CP ranks

### 6.2.1 Core Consistency Plot

In [ ]:
# Perform CP Decomposition and Core Consistency Computation for various ranks

start = time.time()

np.random.seed(32)

num_comps = np.arange(2, 21)
core_cons = []

for num_comp in num_comps:
    _, _, _, core_consistency = cp_with_eff_corcondia(
        users_routes_arr, num_comp)
    core_cons.append([num_comp, core_consistency])

core_cons = pd.DataFrame(core_cons, columns=['num_comp', 'core_consistency'])
display(core_cons.head(1))

elapsed = format_time(time.time() - start)
print(elapsed)

: 

In [ ]:
plt.figure(figsize=(12, 8))

g = sns.lineplot(data=core_cons, x='num_comp', y='core_consistency')

g.axes.set_title('CP Decomposition Rank vs Core Consistency', fontsize=25)

g.set_xlabel('CP Decomposition Rank', fontsize=15)
g.set_ylabel('Core Consistency', fontsize=15)
g.set_xticks(num_comps)
g.set_xticklabels(num_comps)

# label points on the plot
for num_comp, cc in zip(core_cons.num_comp, core_cons.core_consistency):
    plt.text(x=num_comp, y=cc-2, s=cc)

plt.show()

: 

### 6.2.2 T-SNE Plot of Embedding Matrices

In [ ]:
# Create function to visualise both User Embedding and Route Embedding with T-SNE

def embed_visual(profile_df, embed_df, embed_name, perplexity=5):

    if embed_name == 'userEmbed':
        id_col = 'userId'
    elif embed_name == 'routeEmbed':
        id_col = 'Route_id'

    tmp_df = profile_df.join(embed_df.set_index(
        id_col), how='left', on=id_col).copy()

    num_comp = embed_df[embed_name][0].shape[0]

    if num_comp == 2:
        tmp_df['component_0'] = tmp_df.apply(
            lambda x: x[embed_name][0], axis=1)
        tmp_df['component_1'] = tmp_df.apply(
            lambda x: x[embed_name][1], axis=1)
        title = embed_name + \
            ' (CP Rank = ' + str(num_comp) + ') Visualisation without T-SNE'
        xlabel = embed_name + ' component_0'
        ylabel = embed_name + ' component_1'

    elif num_comp > 2:
        tsne_obj = TSNE(n_components=2, verbose=0, perplexity=perplexity,
                        n_iter=1000, random_state=15)

        tsne_results = tsne_obj.fit_transform(
            np.stack(tmp_df[embed_name].to_numpy(), axis=0))

        tsne_user_df = pd.DataFrame(data=tsne_results,
                                    index=embed_df[id_col], columns=['component_0', 'component_1'])

        tmp_df = tmp_df.join(tsne_user_df, how='left', on=id_col)

        title = embed_name + \
            ' (CP Rank = ' + str(num_comp) + ') Visualisation with T-SNE'
        xlabel = embed_name + ' T-SNE component_0'
        ylabel = embed_name + ' T-SNE component_1'

    fig, axes = plt.subplots(1, 3, figsize=(24, 6), dpi=90)
    fig.suptitle(title, fontsize=20)

    for idx, feat in enumerate(['avg_calories', 'avg_speed', 'avg_distance']):
        if embed_name == 'userEmbed':
            sns.scatterplot(x='component_0',
                            y='component_1',
                            palette=sns.color_palette(
                                'rocket_r', as_cmap=True),
                            data=tmp_df,
                            hue=feat,
                            ax=axes[idx])

        elif embed_name == 'routeEmbed':
            sns.scatterplot(x='component_0',
                            y='component_1',
                            palette='colorblind',
                            size=feat,
                            sizes=(2, 200),
                            data=tmp_df,
                            hue='sport',
                            ax=axes[idx],
                            legend='brief')

    for ax in axes:
        ax.set_xlabel(xlabel, fontsize=15)
        ax.set_ylabel(ylabel, fontsize=15)
        ax.legend(loc='lower left',
                  title=ax.get_legend().get_title().get_text(),
                  fontsize='small')

: 

In [ ]:
start = time.time()

num_comps = [2, 11, 13, 16]

for num_comp in num_comps:
    user_matrix, route_matrix, _, _ = cp_with_eff_corcondia(
        users_routes_arr, num_comp)
    user_embed_df = pd.DataFrame(zip(user_list, user_matrix), columns=[
        'userId', 'userEmbed'])

    route_embed_df = data_df[['Route_id']].copy()
    route_embed_df.drop_duplicates(inplace=True, ignore_index=True)
    route_embed_df['routeEmbed'] = route_embed_df.apply(
        lambda x: route_matrix[int(x.Route_id)], axis=1)

    embed_visual(user_profile_df, user_embed_df, 'userEmbed', perplexity=30)
    embed_visual(route_profile_df, route_embed_df, 'routeEmbed', perplexity=5)

elapsed = format_time(time.time() - start)
print(elapsed)

: 

## 6.3 Save Embeddings

In [ ]:
# Optimal number of rank is 13 from the analysis above
num_comp = 13

user_matrix, route_matrix, _, _ = cp_with_eff_corcondia(
    users_routes_arr, num_comp)
user_embed_df = pd.DataFrame(zip(user_list, user_matrix), columns=[
    'userId', 'userEmbed'])

route_embed_df = data_df[['Route_id']].copy()
route_embed_df.drop_duplicates(inplace=True, ignore_index=True)
route_embed_df['routeEmbed'] = route_embed_df.apply(
    lambda x: route_matrix[int(x.Route_id)], axis=1)


# Save Embeddings

userEmbed_file_path = '/content/drive/My Drive/Colab Notebooks/DATA5703/Entity Embedding/userEmbed_tensorD_13.m'
routeEmbed_file_path = '/content/drive/My Drive/Colab Notebooks/DATA5703/Entity Embedding/routeEmbed_tensorD_13.m'

joblib.dump(user_embed_df, userEmbed_file_path)
joblib.dump(route_embed_df, routeEmbed_file_path)

# Load Embeddings

user_embed_df = joblib.load(userEmbed_file_path)
route_embed_df = joblib.load(routeEmbed_file_path)

: 

In [ ]:
user_embed_df.head(3)

: 

In [ ]:
route_embed_df.head(3)

: 

: 